In [20]:
from uuid import uuid4

import pandas as pd
import os

from column_mapping import column_mapping

template_file_path = os.path.abspath("workday_data.xlsx")

master_data_sheet_name="Masterdata_PSteam"
employee_master_sheet_name="Employee Master"
hiring_data_sheet_name="Hiring_data"
lineup_pass_import_sheet_name="LINEUP_PASS_IMPORT"
termination_sheet_name="Termination"
permission_sheet_name="Permission"

master_data_skip_rows = 11
employee_master_skip_rows=11
hiring_data_skip_rows=1
lineup_pass_import_skip_rows=0
termination_skip_rows=10
permission_skip_rows=0

master_data = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                           sheet_name=master_data_sheet_name, skiprows=master_data_skip_rows)

employee_master = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                           sheet_name=employee_master_sheet_name, skiprows=employee_master_skip_rows)

hiring_data = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                           sheet_name=hiring_data_sheet_name,  skiprows=hiring_data_skip_rows)

lineup_pass_import = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                           sheet_name=lineup_pass_import_sheet_name,  skiprows=lineup_pass_import_skip_rows)
termination = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                            sheet_name=termination_sheet_name, skiprows=termination_skip_rows)
permission = pd.read_excel(template_file_path, engine='openpyxl', dtype=object,
                            sheet_name=permission_sheet_name, skiprows=permission_skip_rows)




merged_data_first = pd.merge(
    hiring_data,
    employee_master,
    left_on='Emp id',
    right_on='EMPLOYEE_NUMBER',
    how='left'
)
merged_data_second = pd.merge(
    merged_data_first,
    lineup_pass_import,
    left_on='National ID (SSN/SIN) (National Identifiers)',
    right_on='Số CMND/CCCD:',
    how='left'
)
merged_data_second.to_excel("data_second.xlsx", index=False)
# merged_data_second = pd.merge(
#     merged_data_second,
#     termination,
#     left_on='Emp id',
#     right_on='EMPLOYEE_ID',
#     how='left'
# )

columns = merged_data_second.columns
columns_to_drop = [col for col in columns if col.endswith('_y')]
merged_data_second = merged_data_second.drop(columns=columns_to_drop)
merged_data_second = merged_data_second.rename(columns=lambda x: x.replace('_x', '') if x.endswith('_x') else x)
try:
    template_columns = [col for col in master_data.columns if not str(col).startswith('Unnamed')]
    merged_data_second = merged_data_second.dropna(how='all')
    master_data = master_data.dropna().loc[:, ~master_data.columns.str.contains('^Unnamed')]
    mapped_columns = {}
    for merged_col, master_col in column_mapping.items():
        if merged_col in merged_data_second.columns:
            mapped_columns[master_col] = merged_data_second[merged_col]

    # Tạo DataFrame mới từ dictionary bằng pd.concat
    mapped_df = pd.concat(mapped_columns, axis=1)

    # Lấy danh sách tất cả các cột trong master_data
    master_columns = master_data.columns.tolist()

    # Thêm các cột còn thiếu trong mapped_df và điền giá trị NaN
    for col in template_columns:
        if col not in mapped_df.columns:
            mapped_df[col] = pd.NA

    # Sắp xếp các cột của mapped_df theo thứ tự cột của master_data
    mapped_df = mapped_df[template_columns]

    # Kết hợp dữ liệu từ mapped_df vào master_df
    master_df = pd.concat([master_data, mapped_df], ignore_index=True)
    output_file = 'master_data_updated.xlsx'

    master_df = pd.merge(
        master_df,
        permission,
        on="MSA Client",
        how='left')
    master_df.to_excel(output_file, sheet_name='master_data', index=False)
except Exception as e:
    print(e)

[Errno 13] Permission denied: 'master_data_updated.xlsx'
